# Fuzzy C-Means (FCM)

In [268]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Tratamento dos dados

### Importando o dataset

In [269]:
df = pd.read_csv('Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Verificando as classes

In [270]:
df["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

### Verificando o nome exato das colunas

In [271]:
df.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

### Retirando variáveis inúteis e transformando classes em números

In [272]:
df.drop("Id", axis=1, inplace=True)
df["Species"].replace({"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}, inplace=True)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Verificando a corretude das classes

In [273]:
df["Species"].unique()

array([0, 1, 2], dtype=int64)

### Renomeando a coluna da classe

In [274]:
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Armazendo as classes em uma variável separada

In [275]:
labels = df["Class"].values
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

### Retirando a classe para isolar as variáveis

In [276]:
df.drop("Class", axis=1, inplace=True)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Transformando o dataframe em uma matriz

In [277]:
dados_nn = df.to_numpy()
dados_nn

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

### Normalizando as variáveis por meio de MinMax

In [278]:
scaler = MinMaxScaler()

dados = scaler.fit_transform(dados_nn)

dados

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

### Fuzzyficando o dataset

In [279]:
def crisp_to_fuzzy(y, n_clusters): # transforma o dataset em fuzzy
    fuzzy_labels = np.zeros((len(y), n_clusters)) # cria uma array do dataset preenchida só com zeros
    for i, label in enumerate(y):
        fuzzy_labels[i, label] = 1 # com base na classe, o zero é substituído por um
    return fuzzy_labels

part_dif = crisp_to_fuzzy(labels, 3)
part_dif

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

## Clustering

### Inicialização da matriz de pertinência

In [280]:
def inicializao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Atualização dos centroides

In [281]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

### Atualização da matriz de pertinência

In [282]:
def atualizacao_matriz_pertinencia(dados, centroides, m): # talvez a fórmula para calcular a matriz de pertinência esteja errada
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia / np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Fuzzy C-Means

In [283]:
def fcm(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

### Distância de pertinência

In [284]:
def pertinence_distance(delta_k, delta_k_linha):
    c = delta_k.shape[0] # número de clusters
    # calcula a distância entre δ_k e δ_k' (matrizes de pertinência)
    return (1/c) * np.sum((delta_k - delta_k_linha) ** 2)

### Índice de Rand Difuso

In [285]:
def fuzzy_rand_index(particao1, particao2):
    n = particao1.shape[0]

    total_sum = 0

    for k in range(n):
        for k_linha in range(n):
            if k != k_linha:
                # calcula a métrica para P
                delta_k = particao1[k]
                delta_k_prime = particao1[k_linha]
                EP = 1 - pertinence_distance(delta_k, delta_k_prime)
                
                # calcula a métrica para Q
                delta_k_Q = particao2[k]
                delta_k_prime_Q = particao2[k_linha]
                EQ = 1 - pertinence_distance(delta_k_Q, delta_k_prime_Q)
                
                total_sum += np.abs(EP - EQ) # soma a diferença absoluta entre EP e EQ
    
    denominador = n * (n - 1) / 2
    if denominador == 0:
        raise ValueError
    
    return 1 - (total_sum / denominador)

### Bootstrap

In [286]:
def bootstrap(dados, part_dif, num_clusters, num_trials, num_inits=200):
    indices_rand = []
    n = dados.shape[0] # número de amostras
    for trial in range(num_trials):
        bootstrap_idx = np.random.choice(range(n), size=n, replace=True)
        dados_bootstrap = dados[bootstrap_idx]
        part_dif_bootstrap = part_dif[bootstrap_idx]

        melhor_pertinencia = None
        melhor_objetivo = 1000000000

        for init in range(num_inits):
            centroides, matriz_pertinencia = fcm(dados_bootstrap, num_clusters)
            erro = np.sum(np.linalg.norm(dados_bootstrap - np.dot(matriz_pertinencia, centroides), axis=1))
            if erro < melhor_objetivo:
                melhor_objetivo = erro
                melhor_pertinencia = matriz_pertinencia
        
        idx_rand = fuzzy_rand_index(part_dif_bootstrap, melhor_pertinencia)
        indices_rand.append(idx_rand)
        print(f"Trial {trial} - Índice Rand: {idx_rand}")
    
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)

    return mean_rand_index, std_rand_index

### Definição de parâmetros e execução do método

In [287]:
num_clusters = 3
num_trials = 1000
media_indice_rand, dp_indice_rand = bootstrap(dados, part_dif, num_clusters, num_trials)

print(f"Bootstrap FCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}") # 4 casas decimais
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}") # 4 casas decimais

Trial 0 - Índice Rand: 0.6245615567044127
Trial 1 - Índice Rand: 0.5429902722382672
Trial 2 - Índice Rand: 0.5765304430265201
Trial 3 - Índice Rand: 0.5678972097572104
Trial 4 - Índice Rand: 0.6084345867560392
Trial 5 - Índice Rand: 0.5960822900749108
Trial 6 - Índice Rand: 0.6083453460875076
Trial 7 - Índice Rand: 0.6392836514664315
Trial 8 - Índice Rand: 0.6511139142316138
Trial 9 - Índice Rand: 0.6265752800805923
Trial 10 - Índice Rand: 0.6369790753693241
Trial 11 - Índice Rand: 0.6073643961656929
Trial 12 - Índice Rand: 0.5664289954808828
Trial 13 - Índice Rand: 0.5735044653521582
Trial 14 - Índice Rand: 0.6146234524616037
Trial 15 - Índice Rand: 0.604696901623373
Trial 16 - Índice Rand: 0.65291604085286
Trial 17 - Índice Rand: 0.590865786037412
Trial 18 - Índice Rand: 0.6011115624255804
Trial 19 - Índice Rand: 0.5988058778956346
Trial 20 - Índice Rand: 0.576147368132822
Trial 21 - Índice Rand: 0.5956658627137508
Trial 22 - Índice Rand: 0.6486687668351853
Trial 23 - Índice Rand: 0.